In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import shutil

Extract the zip file:

In [5]:
zip_name = 'fruits-small.zip'
extracted_file = 'fruits-small'

# Make new directory:
os.makedirs(extracted_file, exist_ok = True)

try:
    shutil.unpack_archive(filename= zip_name, extract_dir= extracted_file)
    print('Successfully extacted the file')
except Exception as e:
    print('Error while unpacking the zip file.')

Successfully extacted the file


Steps for Model creation:
0. Data Preprocessing
    i. Image Generators
    ii. Splitting Train and Test data from Generators
1. Model architecting + Prepro
    i. CNN + ANN layers creation
2. Model Compilation
3. Model Training 
4. Model Evaluation
5. Model Testing / Deploy

In [ ]:
# Image Generators:
train_image_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.0)

In [ ]:
# Splitting Test and Train Data:

train_image_data = train_image_generator.flow_from_directory('fruits-small/fruits-small/data/Training',
                                                             batch_size= 20,
                                                             class_mode = 'categorical',
                                                             target_size = (64,64))


test_image_data = test_image_generator.flow_from_directory('fruits-small/fruits-small/data/Validation',
                                                           batch_size= 20,
                                                           class_mode = 'categorical',
                                                           target_size= (64,64))